<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/welcome_to_colab__3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed, Embedding, Bidirectional, Attention, Concatenate, Masking, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ReduceLROnPlateau
import gc
import random
from sklearn.metrics.pairwise import cosine_similarity





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_json("/content/drive/MyDrive/MTGdata/AtomicCards.json")

In [ ]:
def replace_card_name(index, text):
  name_parts = index.split(',')  # Split by comma
  possible_matches = [index]

  # Add individual name parts if comma exists
  if len(name_parts) > 1:
    possible_matches.extend([part.strip() for part in name_parts])

  # Add permutation for names with multiple words before comma
  first_part = name_parts[0].strip()  # Get the part before comma
  first_part_words = first_part.split()  # Split into words
  if len(first_part_words) > 1:
      possible_matches.append(first_part_words[0]) # Add the first word as a match

  # Replace occurrences of possible matches in the text, using word boundaries
  for name in possible_matches:
    text = re.sub(r'\b' + re.escape(name) + r'\b', 'this', text) # Use re.escape and word boundaries
  return text

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'\/', ' ', text)
    text = re.sub(r'\{|\}', ' ', text)
    text = re.sub(r'[^\w\s\+\-]', '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

In [ ]:
text_list = []

for index, row in df.iterrows():
    # Process the indices you're interested in: 0, 1, and 3
    for data_index in [0, 1, 3, 4, 5, 6]:
        try:
            text = row['data'][data_index]['text']
            text = replace_card_name(index, text)
            text = clean_text(text)
            text_list.append('sos ' + text + ' eos')
        except:
            pass  # Silently handle the exception



In [ ]:
# Assuming your list of strings is named 'text_list'
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_list)
total_words = len(tokenizer.word_index) + 1
# Convert text to sequences
input_sequences = tokenizer.texts_to_sequences(text_list)

# Pad sequences to have the same length
max_sequence_length = max([len(seq) for seq in input_sequences])
encoder_input_data = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='post')

# Create a mask with pre-padding
mask = np.zeros_like(encoder_input_data)
for i, seq in enumerate(encoder_input_data):
    mask[i, :len(seq)] = 1  # Mask the actual content, leaving padding tokens unmasked

# Create decoder input data (shifted by one timestep) and decoder target data
decoder_input_data = np.zeros_like(encoder_input_data)
decoder_target_data = np.zeros_like(encoder_input_data)

for i, seq in enumerate(encoder_input_data):
  decoder_input_data[i, 1:] = seq[:-1]  # Shifted by one timestep
  decoder_target_data[i, :-1] = seq[1:]  # Target is the original sequence without the start token

embedding = Embedding(total_words, 256, input_length=max_sequence_length)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
class zerosLayer(tf.keras.Layer):
    def call(self, x):
        return tf.zeros_like(x)

In [ ]:
# Encoder model
encoder_inputs = Input(shape=(max_sequence_length,))
encoder_embedding = embedding(encoder_inputs)
encoder_lstm1 = LSTM(256, return_sequences=True)
encoder_lstm2 = LSTM(256, return_sequences=True, return_state=True)

lstm1_output = encoder_lstm1(encoder_embedding)
encoder_outputs, lstm_h, lstm_c = encoder_lstm2(lstm1_output)

state_h = zerosLayer()(lstm_h)
state_c = lstm_c

encoder_states = [state_h, state_c]


# Define encoder model
encoder_model = Model(encoder_inputs, encoder_states)

In [ ]:
# Decoder model
decoder_inputs = Input(shape=(max_sequence_length,))
decoder_inputs = Dropout(0.2)(decoder_inputs)
decoder_embedding = embedding(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_states = [decoder_state_h, decoder_state_c]

decoder_dense = Dense(total_words, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define decoder model
decoder_model = Model([decoder_inputs] + encoder_states, [decoder_outputs] + decoder_states)

In [ ]:
# Connect the encoder and decoder
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64, epochs=100, validation_split=0.2, sample_weight=mask)

Epoch 1/100
396/396 ━━━━━━━━━━━━━━━━━━━━ 35s 76ms/step - accuracy: 0.8937 - loss: 1.2056 - val_accuracy: 0.9132 - val_loss: 0.4739
Epoch 2/100
396/396 ━━━━━━━━━━━━━━━━━━━━ 30s 76ms/step - accuracy: 0.9188 - loss: 0.4387 - val_accuracy: 0.9255 - val_loss: 0.3957
Epoch 3/100
396/396 ━━━━━━━━━━━━━━━━━━━━ 30s 77ms/step - accuracy: 0.9293 - loss: 0.3678 - val_accuracy: 0.9317 - val_loss: 0.3536
Epoch 4/100
396/396 ━━━━━━━━━━━━━━━━━━━━ 31s 78ms/step - accuracy: 0.9351 - loss: 0.3266 - val_accuracy: 0.9366 - val_loss: 0.3192
Epoch 5/100
396/396 ━━━━━━━━━━━━━━━━━━━━ 31s 78ms/step - accuracy: 0.9390 - loss: 0.2976 - val_accuracy: 0.9389 - val_loss: 0.3012
Epoch 6/100
396/396 ━━━━━━━━━━━━━━━━━━━━ 31s 78ms/step - accuracy: 0.9419 - loss: 0.2772 - val_accuracy: 0.9406 - val_loss: 0.2913
Epoch 7/100
396/396 ━━━━━━━━━━━━━━━━━━━━ 31s 78ms/step - accuracy: 0.9438 - loss: 0.2642 - val_accuracy: 0.9428 - val_loss: 0.2767
Epoch 8/100
396/396 ━━━━━━━━━━━━━━━━━━━━ 31s 78ms/step - accuracy: 0.9460 - loss: 0

In [ ]:
def test_encoder_decoder(encoder_model, decoder_model, tokenizer, text_lst, sample_size=5, max_sequence_length=max_sequence_length):
  """Tests the encoder-decoder network on a small sample of text.

  Args:
    encoder_model: The trained encoder model.
    decoder_model: The trained decoder model.
    tokenizer: The tokenizer used to convert text to sequences.
    text_lst: The list of text strings.
    sample_size: The number of samples to test.
    max_sequence_length: The maximum sequence length used during training.

  Returns:
    None. Prints the original and predicted text for each sample.
  """
  # Get a random sample from text_lst
  sample_indices = random.sample(range(len(text_lst)), sample_size)
  sample_texts = [text_lst[i] for i in sample_indices]

  for text in sample_texts:
    # Encode the input text
    input_seq = tokenizer.texts_to_sequences([text])[0]  # Get the sequence
    input_seq = pad_sequences([input_seq], maxlen=max_sequence_length, padding='post')  # Pad
    states_value = encoder_model.predict(input_seq)

    # Generate the output sequence
    target_seq = np.zeros((1, 1))  # Start with a single token (e.g., start token)
    target_seq[0, 0] = tokenizer.word_index['sos']  # Assuming '<SOS> ' is your start token

    decoded_sentence = ''
    stop_condition = False

    while not stop_condition:
      output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
      sampled_token_index = np.argmax(output_tokens[0, -1, :])  # Sample the next token

      sampled_word = tokenizer.index_word[sampled_token_index] if sampled_token_index != 0 else 'eos'  # Get the word
      decoded_sentence += ' ' + sampled_word

      if sampled_word == 'eos' or len(decoded_sentence.split()) > max_sequence_length:
        stop_condition = True

      target_seq = np.zeros((1, 1))
      target_seq[0, 0] = sampled_token_index
      states_value = [h, c]

    print('Original:', text)
    print('Predicted:', decoded_sentence)
    print('---')

test_encoder_decoder(encoder_model, decoder_model, tokenizer, text_list, sample_size=3)  # Test with 3 samples
